# This is my code for the LSU REU. This WILL NOT run from this notebook. It is merely a copy of the work I did. The code only runs if you are logged onto the LIGO servers, which requires special access.

# Investigation of Noise Transients in the Advanced LIGO Calibration Model

In [ ]:
import gwpy
from matplotlib import use
use('Agg')
from gwpy.timeseries import TimeSeries
from gwpy.timeseries import TimeSeriesDict
from gwpy.plotter import TimeSeriesPlot
from gwpy.plotter import FrequencySeriesPlot
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
%matplotlib inline
mpl.rcParams['agg.path.chunksize'] = 10000

# Import calibration channels, injection signals

In [ ]:
IFO = 'L1'
start_time = 1168605028 
end_time = start_time + 1024

    

main_chanlist = [IFO + ':CAL-DARM_ERR_WHITEN_OUT_DBL_DQ',
                     IFO + ':CAL-PCALY_RX_PD_OUT_DQ',
                     IFO + ':LSC-CAL_LINE_SUM_DQ',
                     IFO + ':SUS-ETMY_L3_CAL_LINE_OUT_DQ',
                     IFO + ':GDS-CALIB_STRAIN']
    
main_chan_data = TimeSeriesDict.fetch(main_chanlist, start_time, end_time, verify=True, verbose=True)


#uncalibrated digital signal from photodetector
llo_DARMERR = main_chan_data[IFO + ':CAL-DARM_ERR_WHITEN_OUT_DBL_DQ']
#photon calibrator injection signal
llo_PCALY = main_chan_data[IFO + ':CAL-PCALY_RX_PD_OUT_DQ']
#control loop injection signal
llo_DARMInj = main_chan_data[IFO + ':LSC-CAL_LINE_SUM_DQ']
#electrostatic drive injection signal
llo_ESD = main_chan_data[IFO + ':SUS-ETMY_L3_CAL_LINE_OUT_DQ']
#calibrated h(t)
llo_STRAIN = main_chan_data[IFO + ':GDS-CALIB_STRAIN']


#need to change the units, this line has the units of * and it causes errors later on if you don't set them to none
llo_DARMInj.override_unit(None) 

# Get ratio channels

In [ ]:

CAL_chanList = [IFO + ':CAL-CS_TDEP_PCALY_LINE1_COMPLEX_RATIO1_REALOUT',
                IFO + ':CAL-CS_TDEP_PCALY_LINE1_COMPLEX_RATIO1_IMAGOUT',
               IFO + ':CAL-CS_TDEP_PCALY_LINE2_COMPLEX_RATIO1_REALOUT',
                IFO + ':CAL-CS_TDEP_PCALY_LINE2_COMPLEX_RATIO1_IMAGOUT',
               IFO + ':CAL-CS_TDEP_PCALY_LINE3_COMPLEX_RATIO1_REALOUT',
                IFO + ':CAL-CS_TDEP_PCALY_LINE3_COMPLEX_RATIO1_IMAGOUT',
               IFO + ':CAL-CS_TDEP_DARM_LINE1_COMPLEX_RATIO1_REALOUT',
                IFO + ':CAL-CS_TDEP_DARM_LINE1_COMPLEX_RATIO1_IMAGOUT',
               IFO + ':CAL-CS_TDEP_SUS_LINE1_COMPLEX_RATIO1_REALOUT',
               IFO + ':CAL-CS_TDEP_SUS_LINE1_COMPLEX_RATIO1_IMAGOUT']



#### Get official ratios ####
CAL_data = TimeSeriesDict.fetch(CAL_chanList, start_time, end_time, verify=True, verbose=True)

PCALY1_ACT_RATIO_R = CAL_data[IFO + ':CAL-CS_TDEP_PCALY_LINE1_COMPLEX_RATIO1_REALOUT']
PCALY1_ACT_RATIO_I = CAL_data[IFO + ':CAL-CS_TDEP_PCALY_LINE1_COMPLEX_RATIO1_IMAGOUT']

PCALY2_ACT_RATIO_R = CAL_data[IFO + ':CAL-CS_TDEP_PCALY_LINE2_COMPLEX_RATIO1_REALOUT']
PCALY2_ACT_RATIO_I = CAL_data[IFO + ':CAL-CS_TDEP_PCALY_LINE2_COMPLEX_RATIO1_IMAGOUT']

PCALY3_ACT_RATIO_R = CAL_data[IFO + ':CAL-CS_TDEP_PCALY_LINE3_COMPLEX_RATIO1_REALOUT']
PCALY3_ACT_RATIO_I = CAL_data[IFO + ':CAL-CS_TDEP_PCALY_LINE3_COMPLEX_RATIO1_IMAGOUT']

DARM1_ACT_RATIO_R = CAL_data[IFO + ':CAL-CS_TDEP_DARM_LINE1_COMPLEX_RATIO1_REALOUT']
DARM1_ACT_RATIO_I = CAL_data[IFO + ':CAL-CS_TDEP_DARM_LINE1_COMPLEX_RATIO1_IMAGOUT']

ESD1_ACT_RATIO_R = CAL_data[IFO + ':CAL-CS_TDEP_SUS_LINE1_COMPLEX_RATIO1_REALOUT']
ESD1_ACT_RATIO_I = CAL_data[IFO + ':CAL-CS_TDEP_SUS_LINE1_COMPLEX_RATIO1_IMAGOUT']

#### Get official TDep Parameters ####

CAL_TDep_chanList = [IFO + ':CAL-CS_TDEP_KAPPA_TST_REAL_INMON',
                    IFO + ':CAL-CS_TDEP_KAPPA_TST_IMAG_INMON',
                    IFO + ':CAL-CS_TDEP_A_REAL_INMON',
                    IFO + ':CAL-CS_TDEP_A_IMAG_INMON',
                    IFO + ':CAL-CS_TDEP_KAPPA_PU_REAL_INMON',
                    IFO + ':CAL-CS_TDEP_KAPPA_PU_IMAG_INMON',
                    IFO + ':CAL-CS_TDEP_KAPPA_C_INMON',
                    IFO + ':CAL-CS_TDEP_F_C_INMON']

CAL_TDep_data = TimeSeriesDict.fetch(CAL_TDep_chanList, start_time, end_time, verify=True, verbose=True)

KT_ACT_R = CAL_TDep_data[IFO + ':CAL-CS_TDEP_KAPPA_TST_REAL_INMON']
KT_ACT_I = CAL_TDep_data[IFO + ':CAL-CS_TDEP_KAPPA_TST_IMAG_INMON']
A_ACT_R = CAL_TDep_data[IFO + ':CAL-CS_TDEP_A_REAL_INMON']
A_ACT_I = CAL_TDep_data[IFO + ':CAL-CS_TDEP_A_IMAG_INMON']
KPU_ACT_R = CAL_TDep_data[IFO + ':CAL-CS_TDEP_KAPPA_PU_REAL_INMON']
KPU_ACT_I = CAL_TDep_data[IFO + ':CAL-CS_TDEP_KAPPA_PU_IMAG_INMON']
KC_ACT = CAL_TDep_data[IFO + ':CAL-CS_TDEP_KAPPA_C_INMON']
FCC_ACT = CAL_TDep_data[IFO + ':CAL-CS_TDEP_F_C_INMON']

# Get EPIC channels

In [ ]:
def getEPICs(start_time, end_time):
    
    EP_chanList = [IFO + ':CAL-CS_TDEP_REF_INVA_CLGRATIO_TST_REAL',
                   IFO + ':CAL-CS_TDEP_REF_INVA_CLGRATIO_TST_IMAG',
                   IFO + ':CAL-CS_TDEP_REF_CLGRATIO_CTRL_REAL',
                   IFO + ':CAL-CS_TDEP_REF_CLGRATIO_CTRL_IMAG',
                   IFO + ':CAL-CS_TDEP_DARM_LINE1_REF_A_USUM_INV_REAL',
                   IFO + ':CAL-CS_TDEP_DARM_LINE1_REF_A_USUM_INV_IMAG',
                   IFO + ':CAL-CS_TDEP_DARM_LINE1_REF_A_TST_REAL',
                   IFO + ':CAL-CS_TDEP_DARM_LINE1_REF_A_TST_IMAG',
                   IFO + ':CAL-CS_TDEP_DARM_LINE1_REF_A_USUM_REAL',
                   IFO + ':CAL-CS_TDEP_DARM_LINE1_REF_A_USUM_IMAG',
                   IFO + ':CAL-CS_TDEP_PCALY_LINE2_REF_C_NOCAVPOLE_REAL',
                   IFO + ':CAL-CS_TDEP_PCALY_LINE2_REF_C_NOCAVPOLE_IMAG',
                   IFO + ':CAL-CS_TDEP_PCALY_LINE2_REF_D_REAL',
                   IFO + ':CAL-CS_TDEP_PCALY_LINE2_REF_D_IMAG',
                   IFO + ':CAL-CS_TDEP_PCALY_LINE2_REF_A_TST_REAL',
                   IFO + ':CAL-CS_TDEP_PCALY_LINE2_REF_A_TST_IMAG',
                   IFO + ':CAL-CS_TDEP_PCALY_LINE2_REF_A_USUM_REAL',
                   IFO + ':CAL-CS_TDEP_PCALY_LINE2_REF_A_USUM_IMAG']
    
    EP_data = TimeSeriesDict.fetch(EP_chanList, start_time, end_time, verify=True, verbose=True)
    
    for chan in EP_chanList:
        if chan == EP_chanList[0]:
            EP1_R = EP_data[chan].median().value
        elif chan == EP_chanList[1]:
            EP1_I = EP_data[chan].median().value
        elif chan == EP_chanList[2]:
            EP2_R = EP_data[chan].median().value
        elif chan == EP_chanList[3]:
            EP2_I = EP_data[chan].median().value
        elif chan == EP_chanList[4]:
            EP3_R = EP_data[chan].median().value
        elif chan == EP_chanList[5]:
            EP3_I = EP_data[chan].median().value
        elif chan == EP_chanList[6]:
            EP4_R = EP_data[chan].median().value
        elif chan == EP_chanList[7]:
            EP4_I = EP_data[chan].median().value
        elif chan == EP_chanList[8]:
            EP5_R = EP_data[chan].median().value
        elif chan == EP_chanList[9]:
            EP5_I = EP_data[chan].median().value
        elif chan == EP_chanList[10]:
            EP6_R = EP_data[chan].median().value
        elif chan == EP_chanList[11]:
            EP6_I = EP_data[chan].median().value
        elif chan == EP_chanList[12]:
            EP7_R = EP_data[chan].median().value
        elif chan == EP_chanList[13]:
            EP7_I = EP_data[chan].median().value
        elif chan == EP_chanList[14]:
            EP8_R = EP_data[chan].median().value
        elif chan == EP_chanList[15]:
            EP8_I = EP_data[chan].median().value
        elif chan == EP_chanList[16]:
            EP9_R = EP_data[chan].median().value
        elif chan == EP_chanList[17]:
            EP9_I = EP_data[chan].median().value        
        
    return EP1_R, EP1_I, EP2_R, EP2_I, EP3_R, EP3_I, EP4_R, EP4_I, EP5_R, EP5_I, EP6_R, EP6_I, EP7_R, EP7_I, EP8_R, EP8_I, EP9_R, EP9_I
 

#### Get EPICs Values ####
print("Getting EPICs values")
[EP1_R, EP1_I, EP2_R, EP2_I, EP3_R, EP3_I, EP4_R, EP4_I, EP5_R, EP5_I, EP6_R, EP6_I, EP7_R, EP7_I, EP8_R, EP8_I, EP9_R, EP9_I] = getEPICs(start_time, end_time)
print("Done.")

# Get line frequencies

In [ ]:
def getCalLines(start_time, end_time):
    
    cal_line_chans = [IFO + ':CAL-CS_TDEP_PCALY_LINE1_COMPARISON_OSC_FREQ', 
                     IFO + ':CAL-CS_TDEP_PCALY_LINE2_COMPARISON_OSC_FREQ',
                     IFO + ':CAL-CS_TDEP_PCALY_LINE3_COMPARISON_OSC_FREQ',
                     IFO + ':CAL-CS_TDEP_DARM_LINE1_COMPARISON_OSC_FREQ',
                     IFO + ':CAL-CS_TDEP_SUS_LINE1_SYNCED_OSC_FREQ']
    
    cal_line_data = TimeSeriesDict.fetch(cal_line_chans, start_time, end_time, verify=True, verbose=True)
    
    for line_chan in cal_line_chans:
        ### If the median and mean vary by more than +- 0.01, give warning
        if np.abs(cal_line_data[line_chan].median().value - cal_line_data[line_chan].mean().value) >= 0.01:
            warnings.warn('Value of %s changes appreciably in the data.' % (line_chan))
            
        if line_chan == cal_line_chans[0]:
            F_PCALY1 = cal_line_data[line_chan].median().value
        elif line_chan == cal_line_chans[1]:
            F_PCALY2 = cal_line_data[line_chan].median().value
        elif line_chan == cal_line_chans[2]:
            F_PCALY3 = cal_line_data[line_chan].median().value
        elif line_chan == cal_line_chans[3]:
            F_DARM1 = cal_line_data[line_chan].median().value
        elif line_chan == cal_line_chans[4]:
            F_ESD1 = cal_line_data[line_chan].median().value
            
    return F_PCALY1, F_PCALY2, F_PCALY3, F_DARM1, F_ESD1

#### Get calibration line frequencies ####
print("Getting calibrational line frequencies..")
[F_PCALY1, F_PCALY2, F_PCALY3, F_DARM1, F_ESD1] = getCalLines(start_time, end_time)


# Create function to perform demodulation

In [ ]:
def demodulate(data, f_demod, bw, sampling_frequency, resample_frequency):
    
    data.override_unit(None)
    
    #bandpassing
    
    data_bp = data.bandpass(f_demod - bw, f_demod + bw,6)
    
    #homodyning
    
    t = np.arange(start_time, end_time, 1./sampling_frequency)
    data_hd = data_bp * np.exp(-2 * np.pi * 1j * f_demod * t)
    
    #lowpass filtering
    data_lp = data_hd.lowpass(bw,4)
    data_resampled = data_lp.resample(resample_frequency)
    
    '''
    asd = data.asd(16,8)
    plot = FrequencySeriesPlot(asd)
    plot1 = plot.gca()
    plot1.set_title('ASD: DARM Error')
    plot1.set_xscale('linear')
    plot1.set_xlim(0,30)
    #plot1.set_ylim(1e-14, 1e-5)
    '''

    
    return 2*data_resampled

    


# Create function for phase shifting signals

In [ ]:
def phaseshift(ratio_I_mine, ratio_R_mine, ratio_I_actual, ratio_R_actual):
    
    #calculate angles for both my data and the actual from channel
    
    phi_mine = np.median(np.arctan(ratio_I_mine/ratio_R_mine).value)
    phi_actual = np.median(np.arctan(ratio_I_actual/ratio_R_actual).value)
    
    dt_phi = phi_actual - phi_mine

    
    sigrot_R = np.cos(dt_phi)*ratio_R_mine - np.sin(dt_phi)*ratio_I_mine 
    sigrot_I = np.sin(dt_phi)*ratio_R_mine + np.cos(dt_phi)*ratio_I_mine
    
    
    #plot to see how shifted ratios match up with actual ratio values from imported channel
    plt.scatter(ratio_R_actual, ratio_I_actual)
    plt.scatter(sigrot_R, sigrot_I)
    plt.show()
    
    print(sigrot_R, "phase shifted real part")
    print(sigrot_I, "phase shifted imaginary part")
    return sigrot_R, sigrot_I
    


# Demodulate signals

In [ ]:
llo_PCALY_f1 = demodulate(llo_PCALY, F_PCALY1, 0.3, 16384, 16) / (F_PCALY1)**2
llo_PCALY_f2 = demodulate(llo_PCALY, F_PCALY2, 0.3, 16384, 16) / (F_PCALY2)**2
llo_PCALY_f3 = demodulate(llo_PCALY, F_PCALY3, 1, 16384, 16) / (F_PCALY3)**2
llo_ESD_f1 = demodulate(llo_ESD, F_ESD1, 0.3, 512, 16)
llo_DARMInj_f1 = demodulate(llo_DARMInj, F_DARM1, 0.3, 16384, 16)

llo_DARMERR_at_PCALY_f1 = demodulate(llo_DARMERR, F_PCALY1, 0.3, 16384, 16)
llo_DARMERR_at_PCALY_f2 = demodulate(llo_DARMERR, F_PCALY2, 0.3, 16384, 16)
llo_DARMERR_at_PCALY_f3 = demodulate(llo_DARMERR, F_PCALY3, 0.3, 16384, 16)
llo_DARMERR_at_ESD_f1 = demodulate(llo_DARMERR, F_ESD1, 0.3, 16384, 16)
llo_DARMERR_at_DARMInj_f1 = demodulate(llo_DARMERR, F_DARM1, 0.3, 16384, 16)

# Calculate new phase shifted ratios

In [ ]:
PCALY1_R, PCALY1_I = phaseshift(np.imag(L1_PCALY1_RATIO), np.real(L1_PCALY1_RATIO), PCALY1_ACT_RATIO_I, PCALY1_ACT_RATIO_R)
PCALY2_R, PCALY2_I = phaseshift(np.imag(L1_PCALY2_RATIO), np.real(L1_PCALY2_RATIO), PCALY2_ACT_RATIO_I, PCALY2_ACT_RATIO_R)
PCALY3_R, PCALY3_I = phaseshift(np.imag(L1_PCALY3_RATIO), np.real(L1_PCALY3_RATIO), PCALY3_ACT_RATIO_I, PCALY3_ACT_RATIO_R)

ESD_R, ESD_I = phaseshift(np.imag(L1_ESD_RATIO), np.real(L1_ESD_RATIO), ESD1_ACT_RATIO_I, ESD1_ACT_RATIO_R)

DARMInj_R, DARMInj_I = phaseshift(np.imag(L1_DARMInj_RATIO), np.real(L1_DARMInj_RATIO), DARM1_ACT_RATIO_I, DARM1_ACT_RATIO_R)

# Calculate the time-dependent calibration parameters

In [ ]:
#scaling factor used for calculating tdep actuation function k_pu
A_f = -(DARMInj_R + DARMInj_I * 1j) * (PCALY1_R + PCALY1_I * 1j)**(-1) * (EP2_R + EP2_I * 1j)


# k_tst, time - dependent correction factors for actuation function
k_tst = (ESD_R + ESD_I * 1j) * (PCALY1_R + PCALY1_I*1j)**(-1) * (EP1_R + EP1_I*1j)
k_pu = (EP3_R + EP3_I * 1J) * (A_f - (k_tst * (EP4_R + EP4_I * 1j)))


#Complex value S
S1 = 1 / (PCALY2_R + PCALY2_I * 1j)
S2 = ((EP7_R + EP7_I * 1j) * ((k_tst * (EP8_R + EP8_I * 1j)) + (k_pu * (EP9_R + EP9_I * 1j))))
S3 = (S1 - S2)**(-1)
S = (1 / (EP6_R + EP6_I * 1j)) * (S3)

#sensing function correction factor
k_c = np.absolute(S)**2 / (np.real(S))

#time-dependent coupled-cavity pole frequency
f_cc = (-np.real(S) / np.imag(S)) * 331.3

#real and imaginary parts for actuation scaling factors
R_k_tst = np.real(k_tst)
I_k_tst = np.imag(k_tst)

R_k_pu = np.real(k_pu)
I_k_pu = np.imag(k_pu)

# Plot the tdep parameters

In [ ]:
## K_tst Real ##

plot = TimeSeriesPlot(R_k_tst, label = 'calculated value')
plot_Rktst = plot.gca()
plot_Rktst.plot(KT_ACT_R, label = 'actual value')
plot_Rktst.set_title('k${_{tst}}$ Real')
plot_Rktst.set_ylabel('Correction value')
plot_Rktst.legend()
plot.show()

In [ ]:
## K_tst Imaginary ###

plot1 = TimeSeriesPlot(I_k_tst, label = 'calculated value')
plot_Iktst = plot1.gca()
plot_Iktst.plot(KT_ACT_I, label = 'actual value')
plot_Iktst.set_title('k${_{tst}}$ Imaginary')
plot_Iktst.set_ylabel('Correction value')
plot_Iktst.legend()
plot.show()

In [ ]:
### K_pu Real ###


plot2 = TimeSeriesPlot(R_k_pu, label = 'calculated value')
plot_Rkpu = plot2.gca()
plot_Rkpu.plot(KPU_ACT_R, label = 'actual value')
plot_Rkpu.set_title('k${_{pu}}$ Real')
plot_Rkpu.set_ylabel('Correction value')
plot_Rkpu.set_ylim(0.6,1.08)
plot_Rkpu.legend()

In [ ]:
### K_pu Imaginary ###


plot3 = TimeSeriesPlot(I_k_pu, label = 'calculated value')
plot_Ikpu = plot3.gca()
plot_Ikpu.plot(KPU_ACT_I, label = 'actual value')
plot_Ikpu.set_title('k${_{pu}}$ Imaginary')
plot_Ikpu.set_ylabel('Correction value')
plot_Ikpu.set_ylim(-0.2,0.1)
plot_Ikpu.legend()

In [ ]:
### K_c ###

plot4 = TimeSeriesPlot(k_c, label = 'calculated value')
plot_kc = plot4.gca()
plot_kc.plot(KC_ACT, label = 'actual value')
plot_kc.set_title('k${_{c}}$')
plot_kc.set_ylim(0.6,1.1)
plot_kc.set_ylabel('Correction value')
plot_kc.legend()
print(k_c)

In [ ]:
## f_cc ##

plot5 = TimeSeriesPlot(f_cc, label = 'calculated value')
plot_fcc = plot5.gca()
plot_fcc.plot(FCC_ACT, label = 'actual value')
plot_fcc.set_ylim(200,600)
plot_fcc.set_title('f${_{cc}}$')
plot_fcc.set_ylabel('Frequency [Hz]')
plot_fcc.legend()



# Amplitude spectral density plot expressed in terms of gravitational wave strain.

In [ ]:
start_time = 1186207218
duration = 1024

chanlist = ['H1:GDS-CALIB_STRAIN', 'L1:GDS-CALIB_STRAIN']

hoft_data = TimeSeriesDict.fetch(chanlist, start_time, start_time + duration, verbose=True)

h1_hoft = hoft_data[chanlist[0]]
l1_hoft = hoft_data[chanlist[1]]



h1_asd = h1_hoft.asd(32, 16)
l1_asd = l1_hoft.asd(32, 16)

plot = h1_asd.plot(label='LIGO-Hanford', color='gwpy:ligo-hanford')
ax = plot.gca()
ax.plot(l1_asd, label='LIGO-Livingston', color='gwpy:ligo-livingston')
ax.set_xlim(10, 2000)
ax.set_ylim(5e-25, 1e-18)
#ax.set_ylabel('Hz^{1/2}')
ax.set_title('H1, L1 Gravitational Wave Strain [h(t)]', fontsize=18)
ax.set_xscale('linear')
ax.legend()
plot.show()